In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Digit Recognizer - CNN in R

Các thành viên:

   - Vũ Đặng Quỳnh Giang
   - Nguyễn Hiền Nhung
   - Nguyễn Tiến Duy
   - Phan Tấn Thịnh




In [ ]:
library(mxnet)

train <- data.matrix(read.csv("../input/digit-recognizer/train.csv", header=T))
test <- data.matrix(read.csv("../input/digit-recognizer/test.csv", header=T))

Có khoảng 4000 quan sát cho mỗi chữ số.

Mỗi hàng có 784 cột (pixel) tạo thành hình ảnh 28x28.

In [ ]:
dim(train[,-1]) 
dim(test)
barplot(table(train[,1]), col=rainbow(10, 0.5), main="n Digits in Train")

Chuyển vị từng ma trận để xoay hình ảnh

In [ ]:
plotTrain <- function(images){
  op <- par(no.readonly=TRUE)
  x <- ceiling(sqrt(length(images)))
  par(mfrow=c(x, x), mar=c(.1, .1, .1, .1))
  
  for (i in images){ 
    m <- matrix(train[i,-1], nrow=28, byrow=TRUE)
    m <- apply(m, 2, rev)
    image(t(m), col=grey.colors(255), axes=FALSE)
    text(0.05, 0.2, col="white", cex=1.2, train[i, 1])
  }
  par(op) 
}

In [ ]:
plotTrain(1:36)

Xóa đi các cột nhãn và chia nhỏ hình.

In [ ]:
train.x <- train[,-1] 
train.y <- train[,1] 
train.x <- t(train.x/255)
test.x <- t(test/255)

Mô hình đầu tiên chỉ sử dụng tầng thứ 3 của mô hình CNN: Fully connected layer.

Sau đó dùng hàm Relu để kích hoạt.

Cuối cùng dùng hàm softMax để tính xác xuất các nhãn.

In [ ]:
m1.data <- mx.symbol.Variable("data")  

m1.fc1 <- mx.symbol.FullyConnected(m1.data, name="fc1", num_hidden=128)
m1.act1 <- mx.symbol.Activation(m1.fc1, name="activation1", act_type="relu")

m1.fc2 <- mx.symbol.FullyConnected(m1.act1, name="fc2", num_hidden=64)
m1.act2 <- mx.symbol.Activation(m1.fc2, name="activation2", act_type="relu")

m1.fc3 <- mx.symbol.FullyConnected(m1.act2, name="fc3", num_hidden=10)
m1.softmax <- mx.symbol.SoftmaxOutput(m1.fc3, name="softMax")

Sử dụng gói MXNet để mô tả mô hình đầu tiên bằng sơ đồ.

In [ ]:
graph.viz(m1.softmax)

Train Model

In [ ]:
log <- mx.metric.logger$new() 
tick <- proc.time() 
mx.set.seed(0)

m1 <- mx.model.FeedForward.create(m1.softmax,  
                                     X = train.x, 
                                     y = train.y, 
                                   
                                     num.round = 1, 
                                     array.batch.size = 100,
                                     array.layout="colmajor",
                                     learning.rate = 0.001,
                                     momentum = 0.95,
                                     eval.metric = mx.metric.accuracy,
                                     initializer = mx.init.uniform(0.07),
                                     epoch.end.callback = mx.callback.log.train.metric(1,log)
)

print(paste("Training took:", round((proc.time() - tick)[3],2),"seconds"))

Đưa ra xác suất của từng số

In [ ]:
m1.preds <- predict(m1, test.x, array.layout = "colmajor")
t(round(m1.preds[,1:5], 2))

In [ ]:
m1.preds.value <- max.col(t(m1.preds)) - 1
m1.preds.value[1:5]

In [ ]:
plotResults <- function(images, preds){
  op <- par(no.readonly=TRUE)
  x <- ceiling(sqrt(length(images)))
  par(mfrow=c(x,x), mar=c(.1,.1,.1,.1))
  
  for (i in images){
    m <- matrix(test[i,], nrow=28, byrow=TRUE)
    m <- apply(m, 2, rev)
    image(t(m), col=grey.colors(255), axes=FALSE)
    text(0.05,0.1,col="red", cex=1.2, preds[i])
  }
  par(op)
}

Đưa ra dự đoán cho 36 hình đầu tiên

In [ ]:
plotResults(1:36, m1.preds.value)

In [ ]:
submission <- data.frame(ImageId=1:ncol(test.x), Label=m1.preds.value)
write.csv(submission, file='kaggle_submission.csv', row.names=FALSE,  quote=FALSE)

In [ ]:
m2.data <- mx.symbol.Variable("data")

#  convolutional layer
m2.conv1 <- mx.symbol.Convolution(m2.data, kernel=c(5,5), num_filter=16)
m2.bn1 <- mx.symbol.BatchNorm(m2.conv1)
m2.act1 <- mx.symbol.Activation(m2.bn1, act_type="relu")
m2.pool1 <- mx.symbol.Pooling(m2.act1, pool_type="max", kernel=c(2,2), stride=c(2,2))
m2.drop1 <- mx.symbol.Dropout(m2.pool1, p=0.5)

# convolutional layer
m2.conv2 <- mx.symbol.Convolution(m2.drop1, kernel=c(3,3), num_filter=32)
m2.bn2 <- mx.symbol.BatchNorm(m2.conv2)
m2.act2 <- mx.symbol.Activation(m2.bn2, act_type="relu")
m2.pool2 <- mx.symbol.Pooling(m2.act2, pool_type="max", kernel=c(2,2), stride=c(2,2))
m2.drop2 <- mx.symbol.Dropout(m2.pool2, p=0.5)
m2.flatten <- mx.symbol.Flatten(m2.drop2)

#  Fully Connected layers
m2.fc1 <- mx.symbol.FullyConnected(m2.flatten, num_hidden=1024)
m2.act3 <- mx.symbol.Activation(m2.fc1, act_type="relu")

m2.fc2 <- mx.symbol.FullyConnected(m2.act3, num_hidden=512)
m2.act4 <- mx.symbol.Activation(m2.fc2, act_type="relu")

m2.fc3 <- mx.symbol.FullyConnected(m2.act4, num_hidden=256)
m2.act5 <- mx.symbol.Activation(m2.fc3, act_type="relu")

m2.fc4 <- mx.symbol.FullyConnected(m2.act5, num_hidden=10)
m2.softmax <- mx.symbol.SoftmaxOutput(m2.fc4)

In [ ]:
graph.viz(m2.softmax)

Chuyển các ma trận train và test về mảng để phù hợp với đầu vào.

In [ ]:
train.array <- train.x
dim(train.array) <- c(28, 28, 1, ncol(train.x))
test.array <- test.x
dim(test.array) <- c(28, 28, 1, ncol(test.x))

In [ ]:
log <- mx.metric.logger$new() 
tick <- proc.time() 
mx.set.seed(0)

m2 <- mx.model.FeedForward.create(m2.softmax, 
                                  X = train.array, 
                                  y = train.y,
                                  num.round = 15, 
                                  array.batch.size = 500,
                                  array.layout="colmajor",
                                  learning.rate = 0.01,
                                  momentum = 0.91,
                                  wd = 0.00001,
                                  eval.metric = mx.metric.accuracy,
                                  initializer = mx.init.uniform(0.07),
                                  epoch.end.callback = mx.callback.log.train.metric(1, log)
)


print(paste("Training took:", round((proc.time() - tick)[3],2),"seconds"))

Sau 20 vòng độ chính xác được cải thiện

Accuracy:97%

In [ ]:
m2.preds <- predict(m2, test.array)
m2.preds.value <- max.col(t(m2.preds)) - 1

plotResults(51:86, m2.preds.value)

submission <- data.frame(ImageId=1:ncol(test.x), Label=m2.preds.value)
write.csv(submission, file='CNN-file.csv', row.names=FALSE, quote=FALSE)